This notebook is a sample code to calculate metrics.
Since There are many ambiguous parts in [official evaluation page](https://www.kaggle.com/c/pku-autonomous-driving/overview/evaluation), this is just my understanding of metrics.

## helper funcs from some kernels

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from math import sqrt, acos, pi, sin, cos
from scipy.spatial.transform import Rotation as R
from sklearn.metrics import average_precision_score
from multiprocessing import Pool

def expand_df(df, PredictionStringCols):
    df = df.dropna().copy()
    df['NumCars'] = [int((x.count(' ')+1)/7) for x in df['PredictionString']]

    image_id_expanded = [item for item, count in zip(df['ImageId'], df['NumCars']) for i in range(count)]
    prediction_strings_expanded = df['PredictionString'].str.split(' ',expand = True).values.reshape(-1,7).astype(float)
    prediction_strings_expanded = prediction_strings_expanded[~np.isnan(prediction_strings_expanded).all(axis=1)]
    df = pd.DataFrame(
        {
            'ImageId': image_id_expanded,
            PredictionStringCols[0]:prediction_strings_expanded[:,0],
            PredictionStringCols[1]:prediction_strings_expanded[:,1],
            PredictionStringCols[2]:prediction_strings_expanded[:,2],
            PredictionStringCols[3]:prediction_strings_expanded[:,3],
            PredictionStringCols[4]:prediction_strings_expanded[:,4],
            PredictionStringCols[5]:prediction_strings_expanded[:,5],
            PredictionStringCols[6]:prediction_strings_expanded[:,6]
        })
    return df

def str2coords(s, names):
    coords = []
    for l in np.array(s.split()).reshape([-1, 7]):
        coords.append(dict(zip(names, l.astype('float'))))
    return coords

def TranslationDistance(p,g, abs_dist = False):
    dx = p['x'] - g['x']
    dy = p['y'] - g['y']
    dz = p['z'] - g['z']
    diff0 = (g['x']**2 + g['y']**2 + g['z']**2)**0.5
    diff1 = (dx**2 + dy**2 + dz**2)**0.5
    if abs_dist:
        diff = diff1
    else:
        diff = diff1/diff0
    return diff

def RotationDistance(p, g):
    true=[ g['pitch'] ,g['yaw'] ,g['roll'] ]
    pred=[ p['pitch'] ,p['yaw'] ,p['roll'] ]
    q1 = R.from_euler('xyz', true)
    q2 = R.from_euler('xyz', pred)
    diff = R.inv(q2) * q1
    W = np.clip(diff.as_quat()[-1], -1., 1.)
    
    # in the official metrics code:
    # https://www.kaggle.com/c/pku-autonomous-driving/overview/evaluation
    #   return Object3D.RadianToDegree( Math.Acos(diff.W) )
    # this code treat θ and θ+2π differntly.
    # So this should be fixed as follows.
    W = (acos(W)*360)/pi
    if W > 180:
        W = 360 - W
    return W

## TP, FP calculation


In [2]:
thres_tr_list = [0.1, 0.09, 0.08, 0.07, 0.06, 0.05, 0.04, 0.03, 0.02, 0.01]
thres_ro_list = [50, 45, 40, 35, 30, 25, 20, 15, 10, 5]

def check_match(idx):
    keep_gt=False
    thre_tr_dist = thres_tr_list[idx]
    thre_ro_dist = thres_ro_list[idx]
    train_dict = {imgID:str2coords(s, names=['carid_or_score', 'pitch', 'yaw', 'roll', 'x', 'y', 'z']) for imgID,s in zip(train_df['ImageId'],train_df['PredictionString'])}
    valid_dict = {imgID:str2coords(s, names=['pitch', 'yaw', 'roll', 'x', 'y', 'z', 'carid_or_score']) for imgID,s in zip(valid_df['ImageId'],valid_df['PredictionString'])}
    result_flg = [] # 1 for TP, 0 for FP
    scores = []
    MAX_VAL = 10**10
    for img_id in valid_dict:
        for pcar in sorted(valid_dict[img_id], key=lambda x: -x['carid_or_score']):
            # find nearest GT
            min_tr_dist = MAX_VAL
            min_idx = -1
            for idx, gcar in enumerate(train_dict[img_id]):
                tr_dist = TranslationDistance(pcar,gcar)
                if tr_dist < min_tr_dist:
                    min_tr_dist = tr_dist
                    min_ro_dist = RotationDistance(pcar,gcar)
                    min_idx = idx
                    
            # set the result
            if min_tr_dist < thre_tr_dist and min_ro_dist < thre_ro_dist:
                if not keep_gt:
                    train_dict[img_id].pop(min_idx)
                result_flg.append(1)
            else:
                result_flg.append(0)
            scores.append(pcar['carid_or_score'])
    
    return result_flg, scores

## map calculation

In [3]:
validation_prediction = '../input/autonomous-driving-validation-data/prediction_for_validation_data.csv'
!head -2 $validation_prediction

ImageId,PredictionString
ID_e91f42875,0.14866611 -0.005883739244167631 -3.1006309186392507 -4.427135573519678 11.727282619890033 68.08457374572754 0.7084255871565703 0.15610172 -3.085861591548371 -3.122734671580144 -5.098906685381513 6.550029311624264 34.46895480155945 0.7480960460554048 0.16366783 0.033130043366090337 -3.1105093398415526 21.997126733844524 7.968561592730302 37.03721761703491 0.7295321301129045


This file is a prediction file for validaion data with same format as submision file.
I used this [kernel](https://www.kaggle.com/hocop1/centernet-baseline) for prediction.

In [4]:
valid_df = pd.read_csv(validation_prediction)
valid_df

,ImageId,PredictionString
0,ID_e91f42875,0.14866611 -0.005883739244167631 -3.1006309186...
1,ID_11c2b877f,0.16128264 -3.1411694866700373 -3.125612245994...
2,ID_75004f45a,0.16297199 -3.1249074856728227 -3.139728123443...
3,ID_d10a385ea,0.1616647 -3.1124009974425286 -3.1339197215908...
4,ID_bc5effdb6,0.13914043 -3.101190067902992 -3.1140565612637...
5,ID_f07bd0eb6,0.15562569 -0.09724600241299564 -3.11218911980...
6,ID_240bf33b7,0.15505455 -3.14086225417293 -3.13713598822290...
7,ID_06117df9b,0.15126768 -3.0219082635424814 -3.112618755670...
8,ID_09658d7d6,0.14164753 -3.1348067085971354 -3.117624715092...
9,ID_5e12ceebe,0.16781648 -0.0004585470162796982 -3.100233471...


In [5]:
valid_df = pd.read_csv(validation_prediction)
expanded_valid_df = expand_df(valid_df, ['pitch','yaw','roll','x','y','z','Score'])
valid_df = valid_df.fillna('')

train_df = pd.read_csv('../input/pku-autonomous-driving/train.csv')
train_df = train_df[train_df.ImageId.isin(valid_df.ImageId.unique())]
# data description page says, The pose information is formatted as
# model type, yaw, pitch, roll, x, y, z
# but it doesn't, and it should be
# model type, pitch, yaw, roll, x, y, z
expanded_train_df = expand_df(train_df, ['model_type','pitch','yaw','roll','x','y','z'])

max_workers = 10
n_gt = len(expanded_train_df)
ap_list = []
p = Pool(processes=max_workers)
for result_flg, scores in p.imap(check_match, range(10)):
    if np.sum(result_flg) > 0:
        n_tp = np.sum(result_flg)
        recall = n_tp/n_gt
        ap = average_precision_score(result_flg, scores)*recall
    else:
        ap = 0
    ap_list.append(ap)
map = np.mean(ap_list)
print('map:', map)

map: 0.05826379915920419


In [6]:
scores

[0.7480960460554048,
 0.7295321301129045,
 0.7084255871565703,
 0.777715720680948,
 0.6041738269664404,
 0.5963370935699751,
 0.5868039470128139,
 0.5527698645755978,
 0.543068795413693,
 0.6619268574124063,
 0.8890155807949937,
 0.7788612978093838,
 0.759001644972492,
 0.7547428422434347,
 0.7353803721015348,
 0.6864580232661144,
 0.6330887537345887,
 0.5433320779783061,
 0.5345800355455937,
 0.5032309156474125,
 0.7758003711049735,
 0.7236448057824587,
 0.6381266907648278,
 0.6059306837513059,
 0.5496539275735354,
 0.812806235171122,
 0.6551903671190131,
 0.6167279447240664,
 0.5823808946257873,
 0.5707296250752311,
 0.5636898121795079,
 0.5440305984006608,
 0.7614255534565981,
 0.6388541791176006,
 0.5093021348279065,
 0.8890344716863295,
 0.8334104312090876,
 0.7706764000536716,
 0.7293527828382548,
 0.6743989813518843,
 0.5908174410641043,
 0.5425945816336573,
 0.732137900240524,
 0.659252940115518,
 0.8812341278984276,
 0.8499168692255951,
 0.7397937436443741,
 0.611397041928863,

In [7]:
result_flg

[0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0]

# note

There are some issues for metrics.

 - [Recall is not considered in Current LB](https://www.kaggle.com/c/pku-autonomous-driving/discussion/116661#latest-673344)
 
 - [Is translation thresholds meter or percentage?](https://www.kaggle.com/c/pku-autonomous-driving/discussion/115630#latest-665163)

 - Official rotation metrics function has issue.(treating θ and θ+2π differntly)
 

I hope these issues will be cleard ASAP.